In [ ]:
#students_manager

In [ ]:
import numpy as np
import pandas as pd
import requests
import pprint
from bs4 import BeautifulSoup
from requests.auth import HTTPBasicAuth
import time
import datetime
import http.client
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('students_info.csv')
df.Udacity_id = df.Udacity_id.astype('int64')
df.Udacity_id = df.Udacity_id.astype('str')

df_old = df.copy()

In [ ]:
# project id for P1 - P4
project_ids_all = {'dand':[330,328,107,355],'band':[324,317,186,551]}
email_add = '***@udacity.com'#输入你的udacity邮箱
password = '***'#输入你的密码

In [ ]:
#signin
url = "https://cn-auth.udacity.com/api/v1/signin"

payload = "{\"email\":\"" + email_add + "\",\"password\":\""+ password +"\"}"
headers = {
    'accept': "application/json",
    'origin': "https://auth.udacity.com",
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
    'content-type': "application/json",
    'referer': "https://auth.udacity.com/sign-in?next=https%3A%2F%2Fstudents.udacity.com",
    'accept-encoding': "gzip, deflate, br",
    'accept-language': "zh-CN,zh;q=0.9,en;q=0.8",
    'cache-control': "no-cache",
    'postman-token': "bdcf883a-15eb-61c8-c200-c6874d7b8118"
    }

response = requests.request("POST", url, data=payload, headers=headers).json()

auth = 'Bearer '+ response['jwt']
auth

In [ ]:
df.head()

In [ ]:
def project_submission_info(student_id):
    '''
    input:student_id
    output: res
    '''
    headers = {"Authorization":auth}
    res = requests.get('https://review-api.udacity.com/api/v1/submissions?udacity_key={}'.format(student_id),headers=headers).json()
    return res

In [ ]:
# several passed info
def project_result(project_history):
    '''
    input:project_history
    output: passed/failed/submitted
    '''
    # print(project_history)
    if len(project_history) == 0:
        return '      '
    elif project_history[0] == 'passed':
        return 'passed'
    elif project_history[0] == 'failed':
        return str(len(project_history)) + ' failed'
    else:
        return str(len(project_history)) + ' submitted'

In [ ]:
# using project_number to output results
def results(project_number):
    '''
    input:project_number,info
    output: project_result
    '''
    project_history = []
    for submission in info:
        if submission['project']['id'] == project_number:
            project_history.append(submission['result'])
    result = project_result(project_history)
    return result

In [ ]:
def update_table(student_id, info, df):  
    '''
    input:student_id,info,df
    output: update_table
    '''
    nanodegree = df[df['Udacity_id'] == student_id]['nanodegrees'].values[0]
    try:
        for num, pro_id in enumerate(project_ids_all[nanodegree]):            
            result = results(pro_id)
            project_columns = "P{}".format(num+1)
            index = df[df['Udacity_id'] == student_id].index
            df.loc[index, project_columns] = result            
    except TypeError:
        print('info not catched for project')
        #print(info)
    

In [ ]:
def last_log_in(student_id):
    '''
    input:student_id
    output: last_log_in
    '''
    nanodegree = df[df['Udacity_id'] == student_id]['nanodegrees'].values[0]
    nanodegrees={'dand':'nd002-cn-basic-vip','band':'nd100-cn-vip','band_try':'nd100-cn-preview'}
    url = "https://classroom-content.udacity.com/api/v1/graphql"
    payload = "{\"query\":\"\\n  query STAN_userQuery ($id: String) {\\n    user (id: $id) {\\n      can_see_professional_profile\\n      can_see_career_portal\\n      settings {\\n        career_portal_selected_nanodegree_track\\n        career_portal_selected_experience_level\\n      }\\n      user_specializations {\\n        nanodegree_id\\n        part_key\\n        specialization_key\\n        status\\n      }\\n      nanodegrees {\\n        id\\n        key\\n        title\\n        version\\n        locale\\n        is_public\\n        is_ready_for_graduation\\n        user_state {\\n          last_viewed_at\\n        }\\n        specializations {\\n          key\\n          title\\n          required_count\\n        }\\n        parts {\\n          key\\n          title\\n          is_public\\n          part_type\\n          modules {\\n            is_public\\n            lessons {\\n              is_public\\n              lab {\\n                id\\n                key\\n                title\\n                is_public\\n                result {\\n                  state\\n                }\\n                user_state {\\n                  completed_at\\n                }\\n              }\\n              project {\\n                id\\n                key\\n                title\\n                is_public\\n                rubric_id\\n                project_state{\\n                  state\\n                  submissions{\\n                    status\\n                    result\\n                    url\\n                  }\\n                }\\n              }\\n            }\\n          }\\n        }\\n      }\\n      graduated: nanodegrees(is_graduated: true) {\\n        id\\n        key\\n        title\\n        version\\n        specializations {\\n          key\\n          title\\n          required_count\\n        }\\n        parts {\\n          key\\n          title\\n          is_public\\n          part_type\\n          modules {\\n            lessons {\\n              lab {\\n                id\\n                key\\n                title\\n                result {\\n                  state\\n                }\\n                user_state {\\n                  completed_at\\n                }\\n              }\\n              project {\\n                id\\n                key\\n                title\\n                rubric_id\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n\",\"variables\":{\"id\":\""+str(student_id)+"\"}}"
    headers = {
        'authorization':auth,
        'content-type': "application/json"
    }
    print(df[df['Udacity_id'] == student_id].student_name.values[0])
    res = requests.request("POST", url, data=payload, headers=headers).json()
    #print(res)
    try:
        for i in res['data']['user']['nanodegrees']:
            #pprint.pprint(i)
            if i['key'] == nanodegrees[nanodegree]:
                try:
                    last_login = i['user_state']['last_viewed_at'].split('T')[0]
                    #print(last_login)
                    last_view = pd.datetime.strptime(last_login, '%Y-%m-%d')
                    diff = (today_datetime - last_view).days
                    if diff > 3:
                        print(tubiao*diff,diff,'天没学习了。。。')
                    return last_login
                except AttributeError:
                    print('Not Begin Yet')
                    return 'Not Begin Yet'
    except TypeError:
        print('info not catched for login')

In [ ]:
def update_login(student_id, last_login, df):
    '''
    input:student_id,lastlogin,df
    output: update_login
    '''
    try:
        index = df[df['Udacity_id'] == student_id].index
        df.loc[index, 'Last_login_old'] = df.loc[index, 'Last_login_new']
        df.loc[index, 'Last_login_new'] = last_login
    except KeyError:
        print('info not catched for login')

In [ ]:
def update_gap():
    #几天未登录

    today = time.strftime("%Y-%m-%d", time.localtime())
    today_datetime = pd.datetime.strptime(today, '%Y-%m-%d')
    today_datetime 

    diff_list = []
    for i in df.Last_login_new:
        if i != 'Not Begin Yet' and i!=None:
            last_view = pd.datetime.strptime(i, '%Y-%m-%d')
            diff = (today_datetime - last_view).days
        else:
            diff = ''
        diff_list.append(diff)
    df['Gap'] = diff_list

In [ ]:
done_list = []
count = 0
count_sum=len(df.Udacity_id)
today = time.strftime("%Y-%m-%d", time.localtime())
today_datetime = pd.datetime.strptime(today, '%Y-%m-%d')
tubiao ='📆'
#3天以内学习过的不显示日历图标；
#4天以上没学习的则显示图标，一个图标代表一天

for id in df.Udacity_id:
    count +=1
    print('********{}/{}********'.format(count,count_sum))
    info = project_submission_info(id)
    #print(info)
    update_table(id, info, df)

    info2 = last_log_in(id)
    #print(info2)
    update_login(id, info2, df)
    done_list.append(id)
    time.sleep(1)
update_gap()

In [ ]:
left = len(df) - len(done_list)
print(' 完成更新人数：',len(done_list),'\n','剩余更新人数：', left)

In [ ]:
# ready to continue if still anyone's not completed
id_left = df.Udacity_id[len(done_list):]
if len(id_left) == 0:
    print('All done.')
else:
    for id in id_left:
        print('********{}/{}********'.format(count,count_sum))
        info = project_submission_info(id)
        update_table(id, info, df)

        info2 = last_log_in(id)
        update_login(id, info2, df)
        count +=1
        done_list.append(id)

In [ ]:
#有更新的名单
print('\n学员项目新动态：')
index_ =[]
for index in df.index:
    for col in ['P1','P2','P3','P4']:
        if df_old.loc[index,col] != df.loc[index,col]:
            index_.append(index)
df_new = df.loc[index_,]
df_new

In [ ]:
#按日期存档
today = str(datetime.date.today()).replace("-","")
df.to_csv('students_info{}.csv'.format(today),index=False,encoding="utf_8_sig")

#最新文件更新

df.to_csv('students_info.csv',index=False,encoding="utf_8_sig")

# Analysis
Starting

In [ ]:
df = pd.read_csv('students_info.csv')
#df.Udacity_id = df.Udacity_id.astype('int64')
stats_pf = df
#stats_pf.head()

In [ ]:
stats_df = df.copy()
for i in range(1,5):
    column = 'P{}'.format(i)
    stats_df[column] = stats_df[column].apply(lambda x: 1 if x =='passed' else 0)
#stats_pf.head()

In [ ]:
pd.pivot_table(stats_df,index=['nanodegrees','ClassID'],values=['P1','P2','P3','P4'],aggfunc=[lambda x:format(np.mean(x),'.0%')])

In [ ]:
print('\n项目一没有通过的同学：')
stats_df[stats_df['P1'] == 0][['ClassID','student_name','P1','P2','Gap']]